# Sentiment Analysis with Hugging Face

Hugging Face is an open-source and platform provider of machine learning technologies. You can use install their package to access some interesting pre-built models to use them directly or to fine-tune (retrain it on your dataset leveraging the prior knowledge coming with the first training), then host your trained models on the platform, so that you may use them later on other devices and apps.

Please, [go to the website and sign-in](https://huggingface.co/) to access all the features of the platform.

[Read more about Text classification with Hugging Face](https://huggingface.co/tasks/text-classification)

The Hugging face models are Deep Learning based, so will need a lot of computational GPU power to train them. Please use [Colab](https://colab.research.google.com/) to do it, or your other GPU cloud provider, or a local machine having NVIDIA GPU.

## Application of Hugging Face Text classification model Fune-tuning

Find below a simple example, with just `10 epochs of fine-tuning`.

Read more about the fine-tuning concept : [here](https://deeplizard.com/learn/video/5T-iXNNiwIs#:~:text=Fine%2Dtuning%20is%20a%20way,perform%20a%20second%20similar%20task.)

In [1]:
%%capture

!pip install transformers
!pip install accelerate -U
!pip install datasets
!pip install huggingface_hub

In [2]:
%%capture

import torch
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from transformers import pipeline
from datasets import load_dataset
import nltk
nltk.download('punkt')
from torch import nn
from transformers import TrainingArguments
from transformers import Trainer
##others
import warnings
warnings.filterwarnings("ignore")
import os
os.environ["WANDB_DISABLED"] = "true"

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 50)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing Dataset from Google Drive**

In [4]:
data_path= "/content/drive/MyDrive/Colab Notebooks/NLP/Transformed_copy.csv"

In [5]:
##reading data
data= pd.read_csv(data_path)

In [6]:
data.head()

,Unnamed: 0,safe_tweet,label,agreement
0,0,amp big homie meanboy stegman st,0.0,1.0
1,1,im thinking devoting career proving autism isn...,1.0,1.0
2,2,vaccines vaccinate child,-1.0,1.0
3,3,mean immunize kid something wont secretly kill...,-1.0,1.0
4,4,thanks catch performing la nuit nyc st ave sho...,0.0,1.0


In [7]:
#Check for null values

data.isna().sum()

Unnamed: 0     0
safe_tweet    29
label          0
agreement      0
dtype: int64

In [8]:
data[data["safe_tweet"].isnull()]

,Unnamed: 0,safe_tweet,label,agreement
444,444,NaN,0.0,1.000000
1523,1523,NaN,0.0,1.000000
2155,2155,NaN,0.0,1.000000
2515,2515,NaN,0.0,1.000000
3062,3062,NaN,0.0,0.666667
3204,3204,NaN,0.0,1.000000
3819,3819,NaN,1.0,0.666667
4631,4631,NaN,0.0,1.000000
4638,4638,NaN,0.0,1.000000
4770,4770,NaN,0.0,1.000000


In [9]:
##drop Unnamed column and missing values to facilitate analysis
data= data.drop("Unnamed: 0", axis=1)

data= data.dropna()


In [10]:
data

,safe_tweet,label,agreement
0,amp big homie meanboy stegman st,0.0,1.000000
1,im thinking devoting career proving autism isn...,1.0,1.000000
2,vaccines vaccinate child,-1.0,1.000000
3,mean immunize kid something wont secretly kill...,-1.0,1.000000
4,thanks catch performing la nuit nyc st ave sho...,0.0,1.000000
...,...,...,...
9994,living time sperm used waste jenny mccarthy be...,1.0,1.000000
9995,spite measles outbreaks judge mi threatens put...,1.0,0.666667
9996,interesting trends child immunization oklahoma...,0.0,1.000000
9997,cdc says measles highest levels decades return...,0.0,1.000000


In [11]:
#Ensuring there are no null values

data.isna().sum()

safe_tweet    0
label         0
agreement     0
dtype: int64

In [12]:
# Change tweet rows to tuples  to conform to the standard

data['safe_tweet'] = data['safe_tweet'].apply(lambda tweet: tuple(tweet.split(),))

 **Data Splitting**

In [13]:
train, eval= train_test_split(data, test_size= 0.2, stratify= data["label"])

In [14]:
train.head()

,safe_tweet,label,agreement
6363,"(average, people, complain, live, longer, rele...",0.0,1.000000
4402,"(outbreak, ohio, adds, year, high, measles, ca...",0.0,0.666667
3135,"(go, vaccinations, college, im, deathly, scare...",-1.0,0.333333
6346,"(orgasms, increase, levels, youth, hormone, dh...",0.0,0.666667
4701,"(duh, takes, dead, virus, flu, make, flu, vacc...",1.0,0.666667


In [15]:
eval.head()

,safe_tweet,label,agreement
4101,"(needs, evaluated, basis, disease, prevents, r...",0.0,0.666667
2431,"(matter, old, will, always, scared, blood, ext...",-1.0,1.000000
9326,"(cdc, outbreak, growing, least, people, states...",1.0,1.000000
1119,"(mmr,)",0.0,1.000000
4150,"(never, boring, day, two, days, school, sent, ...",0.0,1.000000


In [16]:
print(f"new dataframe shapes: train is {train.shape}, eval is {eval.shape}")

new dataframe shapes: train is (7976, 3), eval is (1994, 3)


In [17]:
#saving the train and eval data to csv

train.to_csv("/content/train.csv")
eval.to_csv("/content/eval.csv")

 **Loading Datasets**

In [18]:
dataset= load_dataset("csv", data_files={"train":"train.csv", "eval":"eval.csv" }, encoding= "ISO-8859-1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [19]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'safe_tweet', 'label', 'agreement'],
        num_rows: 7976
    })
    eval: Dataset({
        features: ['Unnamed: 0', 'safe_tweet', 'label', 'agreement'],
        num_rows: 1994
    })
})

**Tokenization**

In [20]:
#create an instance for tokenizer
tokenizer= AutoTokenizer.from_pretrained("roberta-base")


## Preprocessing Data

In [21]:
## changing labels to 0,1,2  from the initial labels -1, 0, 1
def transform_labels(input):
  label= input["label"]
  num =0

  if label== -1:
    num= 0
  elif label== 0:
    num =1
  elif label == 1:
    num = 2
  return {"labels": num}

def tokenize(example):
  return tokenizer(example["safe_tweet"], padding= "max_length", truncation=True, return_tensors= "pt")


In [22]:
## Converting tweets to tokens for the model to work with
dataset= dataset.map(tokenize, batched= True)

Map:   0%|          | 0/7976 [00:00<?, ? examples/s]

Map:   0%|          | 0/1994 [00:00<?, ? examples/s]

In [23]:
## eliminating features that are not needed for the analysis
remove_columns= ['Unnamed: 0', 'safe_tweet', 'label', 'agreement']
dataset = dataset.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/7976 [00:00<?, ? examples/s]

Map:   0%|          | 0/1994 [00:00<?, ? examples/s]

In [24]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7976
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1994
    })
})

**Modelling**

In [25]:
#loading  model and creating an instance for the classes
model= AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels= 3)

In [26]:
from transformers import AutoModel
Roberta = 'roberta-base'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels= 3). to(device)

#model = AutoModel.from_pretrained(Roberta).to(device)

In [27]:
model_name = f"{Roberta}-Roberta-Model"

In [28]:
#Setting batch size
batch_size= 16


In [29]:

training_args = TrainingArguments( output_dir= model_name,
   num_train_epochs=10, load_best_model_at_end=True,evaluation_strategy="steps",save_strategy="steps",push_to_hub=True

)

In [30]:
##setting a shuffle seed to avoid randomization at each rerun
train_dataset= dataset['train'].shuffle(seed=10)
eval_dataset= dataset['eval'].shuffle(seed=10)

In [31]:
#Connecting to huggingface

from huggingface_hub import notebook_login

notebook_login()

In [34]:
# using f1-score as a metric score because of imbalance in the dataset

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average="weighted")
  return {"f1": f1}

In [35]:
#trainer= Trainer(
    #model= model,
      #args= training_args,
      #train_dataset= train_dataset,
      #eval_dataset= eval_dataset,
      #tokenizer= tokenizer,
      #compute_metrics=compute_metrics

#)

In [36]:
#training the model

#trainer.train()

Step,Training Loss,Validation Loss,F1
500,0.861200,0.783724,0.639426
1000,0.796400,0.759222,0.667190
1500,0.887000,0.781808,0.667717
2000,0.824900,0.849169,0.653981
2500,0.791600,0.772087,0.673598
3000,0.798800,0.839090,0.623037
3500,0.869100,0.810436,0.666499
4000,0.847300,0.783435,0.668654
4500,0.779800,0.758234,0.669663
5000,0.788600,0.800404,0.649913


TrainOutput(global_step=9970, training_loss=0.8425987373742321, metrics={'train_runtime': 9187.0528, 'train_samples_per_second': 8.682, 'train_steps_per_second': 1.085, 'total_flos': 2.098592619798528e+16, 'train_loss': 0.8425987373742321, 'epoch': 10.0})

In [37]:

#trainer.evaluate()

{'eval_loss': 0.7452447414398193,
 'eval_f1': 0.6666149878090573,
 'eval_runtime': 61.8377,
 'eval_samples_per_second': 32.246,
 'eval_steps_per_second': 4.043,
 'epoch': 10.0}

In [38]:
#  ##pushing the trained model to hugginface

#trainer.push_to_hub()

'https://huggingface.co/Enyonam/roberta-base-Roberta-Model/tree/main/'

In [39]:
# creating class weights
class_weights= (1-(data["label"].value_counts().sort_index() /len(data))).values
class_weights

array([0.89618857, 0.50992979, 0.59388164])

In [40]:
#saving class weights to device  gpu
class_weights= torch.from_numpy(class_weights).float().to(device)

In [49]:
##creating a custom class to enable the classweight
class WeightedLossTrainer(Trainer):

   def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]
        inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits.float()
        labels = labels
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [50]:
#creating an instance for the training arguments with weights
#training_args = TrainingArguments(
   #output_dir= model_name,
   #num_train_epochs=10, load_best_model_at_end=True, weight_decay=0.01, evaluation_strategy="steps",save_strategy="steps",push_to_hub=True

#)

In [51]:
#train_dataset= dataset['train'].shuffle(seed=10)
#eval_dataset= dataset['eval'].shuffle(seed=10)



In [52]:
##creating an instance for the custom trainer
trainer = WeightedLossTrainer(
      model= model,
      args= training_args,
      train_dataset= train_dataset,
      eval_dataset= eval_dataset,
      tokenizer= tokenizer,
      compute_metrics=compute_metrics )

In [ ]:
#Training the model with the class weights
trainer.train()

Step,Training Loss,Validation Loss,F1
500,0.886600,0.874835,0.644094
1000,0.836600,0.837708,0.670684
1500,0.843300,0.852078,0.668220
2000,0.863000,0.865030,0.664313
2500,0.861100,0.833676,0.666330
3000,0.877900,1.013143,0.322249


In [ ]:

trainer.evaluate()




In [ ]:
#Pushing the model to the hub
trainer.push_to_hub()
